In [ ]:
import ast
import matplotlib.pyplot as plt 
import numpy as np
import itertools
import random
import seaborn as sns
from scipy.spatial.distance import cdist
import gensim
from gensim.models import Word2Vec,FastText
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import tqdm
import pandas as pd
from itertools import combinations
from itertools import product

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# my own code
import prep_data_functions
import embeddings_functions

# Experiment 2

In [ ]:
document_length = 3
experimental = prep_data_functions.ExperimentalDF(num_documents = 10, document_length=document_length)
corpus2_df = experimental.define_first_df()
print(corpus2_df)

In [ ]:
np.random.seed(3)
docs = list(set(corpus2_df.Document.values))
np.random.shuffle(docs)
half_len = len(docs) // 2

train_docs = docs[:half_len]
test_docs = docs[half_len:]

words_from_train = corpus2_df[corpus2_df["Document"].isin(train_docs)]
df_train = experimental.aggregate_documents(words_from_train)
df_train = experimental.tokenize_data('document', df_train)

words_from_test = corpus2_df[corpus2_df["Document"].isin(test_docs)]
df_test = experimental.aggregate_documents(words_from_test)
df_test = experimental.tokenize_data('document', df_test)

In [ ]:
test_docs

In [ ]:
train_docs

In [ ]:
df_test

In [ ]:
df_train

In [ ]:
df1 = pd.DataFrame(corpus2_df)
combinations_list = list(product(df1['Word'], repeat=2))
df2 = pd.DataFrame(combinations_list, columns=['Word1', 'Word2'])
df2['Document1'] = df2['Word1'].map(df1.set_index('Word')['Document'])
df2['Document2'] = df2['Word2'].map(df1.set_index('Word')['Document'])
df2 = df2[df2["Document1"]!=df2["Document2"]]
df2 = df2[["Word1","Word2"]]
df2.columns = ["First","Last"]
df2

In [ ]:
unique_pairs_train = pd.DataFrame(df_train, columns=['First',"Middle", 'Last']).drop_duplicates()
unique_pairs_test = pd.DataFrame(df_test, columns=['First', "Middle",'Last']).drop_duplicates()
    
unique_pairs_train["join_middle_first"] = unique_pairs_train["First"]+unique_pairs_train["Middle"]
unique_pairs_train["join_middle_last"] = unique_pairs_train["Middle"]+unique_pairs_train["Last"]
unique_pairs_train["join_first_last"] = unique_pairs_train["First"]+unique_pairs_train["Last"]
unique_pairs_test["join_middle_first"] = unique_pairs_test["First"]+unique_pairs_test["Middle"]
unique_pairs_test["join_middle_last"] = unique_pairs_test["Middle"]+unique_pairs_test["Last"]
unique_pairs_test["join_first_last"] = unique_pairs_test["First"]+unique_pairs_test["Last"] 

In [ ]:
unique_pairs_test

In [ ]:
unique_pairs_train

## Skip gram model - low frequency vs high frequency

In [ ]:
test_docs_indexes=[int(test_docs[item].split("c")[1]) for item in range(0,len(test_docs))]

In [ ]:
avg_train_list = []
avg_test_list = []
avg_df2_list = []
corpus_size_dupl = []
corpus_size = []
for cnt_of_dupl in tqdm.tqdm(list(range(1, 30000, 500))):
    dict_index_count = {test_docs_indexes[item]: cnt_of_dupl for item in range(0,len(test_docs_indexes))}
    corpus_size_dupl.append(cnt_of_dupl)
    corpus_pokus_t = experimental.create_experiment(dict_index_count=dict_index_count, df=corpus2_df)
    corpus_size.append(len(corpus_pokus_t))
    word_embeddings = embeddings_functions.Word2VecEmbeddings(corpus_pokus_t,"experiment2")
    words = list(np.array(corpus_pokus_t).ravel())
    embeddings = word_embeddings.train_and_get_emb(words=words, window=1, min_count=1, seed=1, sg=1, vector_size=100,sample=None,epochs=5,norm=True)
    similarity = embeddings_functions.Similarity2(embeddings_df=embeddings)
    df_cosine = similarity.get_cosine_similarity_of_all_words()

    df_cosine["join_middle_first"] = df_cosine["First"] + df_cosine["Last"]
    df_cosine["join_middle_last"] = df_cosine["First"] + df_cosine["Last"]
    df_cosine["join_first_last"] = df_cosine["First"] + df_cosine["Last"]
    
    sim_vals_train = df_cosine.merge(unique_pairs_train, on=['join_middle_first'], how='inner')["similarity"].tolist()
    sim_vals_2_train = df_cosine.merge(unique_pairs_train, on=['join_middle_last'], how='inner')["similarity"].tolist()
    sim_list_train = sim_vals_train + sim_vals_2_train
    
    sim_vals_test = df_cosine.merge(unique_pairs_test, on=['join_middle_first'], how='inner')["similarity"].tolist()
    sim_vals_2_test = df_cosine.merge(unique_pairs_test, on=['join_middle_last'], how='inner')["similarity"].tolist()
    sim_list_test = sim_vals_test + sim_vals_2_test

    avg_test = np.average(sim_list_test)
    avg_test_list.append(avg_test)
    
    avg_train = np.average(sim_list_train)
    avg_train_list.append(avg_train)
    
    avg_df2 = np.average(df_cosine.merge(df2, on = ["First","Last"])["similarity"].tolist())
    avg_df2_list.append(avg_df2)
    

### Fast Text

In [ ]:
avg_train_list2 = []
avg_test_list2 = []
avg_df2_list2 = []
corpus_size_dupl2 = []
corpus_size2 = []
for cnt_of_dupl in tqdm.tqdm(list(range(1, 30000, 500))):
#for cnt_of_dupl in tqdm.tqdm(list(range(1, 10, 2))):
    dict_index_count = {test_docs_indexes[item]: cnt_of_dupl for item in range(0,len(test_docs_indexes))}
    corpus_size_dupl2.append(cnt_of_dupl)
    corpus_pokus_t = experimental.create_experiment(dict_index_count=dict_index_count, df=corpus2_df)
    corpus_size2.append(len(corpus_pokus_t))
    word_embeddings = embeddings_functions.FastTextEmbeddings(corpus_pokus_t,"experiment2_ft")
    words = list(np.array(corpus_pokus_t).ravel())
    embeddings = word_embeddings.train_and_get_emb(words=words, window=1, min_count=1, seed=1, sg=1, vector_size=100,sample=None,epochs=5,norm=True)
    similarity = embeddings_functions.Similarity2(embeddings_df=embeddings)
    df_cosine = similarity.get_cosine_similarity_of_all_words()

    df_cosine["join_middle_first"] = df_cosine["First"] + df_cosine["Last"]
    df_cosine["join_middle_last"] = df_cosine["First"] + df_cosine["Last"]
    df_cosine["join_first_last"] = df_cosine["First"] + df_cosine["Last"]
    
    sim_vals_train = df_cosine.merge(unique_pairs_train, on=['join_middle_first'], how='inner')["similarity"].tolist()
    sim_vals_2_train = df_cosine.merge(unique_pairs_train, on=['join_middle_last'], how='inner')["similarity"].tolist()
    sim_list_train = sim_vals_train + sim_vals_2_train
    
    sim_vals_test = df_cosine.merge(unique_pairs_test, on=['join_middle_first'], how='inner')["similarity"].tolist()
    sim_vals_2_test = df_cosine.merge(unique_pairs_test, on=['join_middle_last'], how='inner')["similarity"].tolist()
    sim_list_test = sim_vals_test + sim_vals_2_test

    avg_test = np.average(sim_list_test)
    avg_test_list2.append(avg_test)
    
    avg_train = np.average(sim_list_train)
    avg_train_list2.append(avg_train)
    
    avg_df2 = np.average(df_cosine.merge(df2, on = ["First","Last"])["similarity"].tolist())
    avg_df2_list2.append(avg_df2) 

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(avg_train_list, label='Words once appeared in same context')
plt.plot(avg_test_list, label='Words frequently appeared togather')
plt.plot(avg_df2_list, label='Words never appeared togather')
plt.legend()  
plt.show()

plt.figure(figsize=(6, 6))
plt.plot(avg_train_list2, label='Words once appeared in same context')
plt.plot(avg_test_list2, label='Words frequently appeared togather')
plt.plot(avg_df2_list2, label='Words never appeared togather')
plt.legend()  
plt.show()

In [ ]:
df_cosine["similarity"].hist()

In [ ]:
df_cosine.sort_values("similarity")

In [ ]:
df_cosine.sort_values("similarity",ascending=False)